In [1]:
import numpy as np
import pyscf

In [2]:

# ==> Define function to diagonalize F <==
def diag_F(F, norb):
    F_p = A.dot(F).dot(A)
    e, C_p = np.linalg.eigh(F_p)
    C = A.dot(C_p)
    C_occ = C[:, :norb]
    D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)
    return (C, D)

In [3]:

# ==> Build DIIS Extrapolation Function <==
def diis_xtrap(F_list, DIIS_RESID):
    # Build B matrix
    B_dim = len(F_list) + 1
    B = np.empty((B_dim, B_dim))
    B[-1, :] = -1
    B[:, -1] = -1
    B[-1, -1] = 0
    for i in range(len(F_list)):
        for j in range(len(F_list)):
            B[i, j] = np.einsum('ij,ij->', DIIS_RESID[i], DIIS_RESID[j], optimize=True)

    # Build RHS of Pulay equation 
    rhs = np.zeros((B_dim))
    rhs[-1] = -1
      
    # Solve Pulay equation for c_i's with NumPy
    coeff = np.linalg.solve(B, rhs)
      
    # Build DIIS Fock matrix
    F_DIIS = np.zeros_like(F_list[0])
    for x in range(coeff.shape[0] - 1):
        F_DIIS += coeff[x] * F_list[x]
    
    return F_DIIS

In [5]:
#geometry
from pyscf import gto
mol=gto.Mole()
mol.atom='''
    O
    H 1 1.1
    H 1 1.1 2 104
'''#Z-matrix input format
mol.basis='cc-pvdz'
mol.symmetry = True
mol.build()

In [12]:
print(mol.nelec)
print(np.shape(mol.nelec))

(5, 5)
(2,)


In [6]:
# ==> Set default program options <==
# Maximum SCF iterations
MAXITER = 40
# Energy convergence criterion
E_conv = 1.0e-6
D_conv = 1.0e-3

In [15]:
S=mol.intor('int1e_ovlp')
nbf = S.shape[0]
nalpha=mol.nelec[0]
nbeta=mol.nelec[1]
ndocc=min(nalpha,nbeta)
print('Number of basis functions: %d' % (nbf))
print('Number of singly occupied orbitals: %d' % (abs(nalpha - nbeta)))
print('Number of doubly occupied orbitals: %d' % (ndocc))


Number of basis functions: 24
Number of singly occupied orbitals: 0
Number of doubly occupied orbitals: 5


In [16]:
#Build ERI Tensor
I = mol.intor('int2e')
print(np.shape(I))
#Build core Hamiltonian
T = mol.intor('int1e_kin')
V = mol.intor('int1e_nuc')
H = T + V

(24, 24, 24, 24)


In [18]:
#symmetric orthogonalization
from scipy.linalg import fractional_matrix_power
A = fractional_matrix_power(S, -0.5)

In [19]:
# ==> Build alpha & beta CORE guess <==
Ca, Da = diag_F(H, nalpha)
Cb, Db = diag_F(H, nbeta)

# Get nuclear repulsion energy
E_nuc = mol.energy_nuc()

In [20]:
# ==> Pre-Iteration Setup <==
# SCF & Previous Energy
SCF_E = 0.0
E_old = 0.0

In [21]:
# Trial & Residual Vector Lists -- one each for alpha & beta
F_list_a = []
F_list_b = []
R_list_a = []
R_list_b = []

# ==> UHF-SCF Iterations <==
print('==> Starting SCF Iterations <==\n')

# Begin Iterations
for scf_iter in range(1, MAXITER+1):
    # Build Fa & Fb matrices
    Ja = np.einsum('pqrs,rs->pq', I, Da, optimize=True)
    Jb = np.einsum('pqrs,rs->pq', I, Db, optimize=True)
    Ka = np.einsum('prqs,rs->pq', I, Da, optimize=True)
    Kb = np.einsum('prqs,rs->pq', I, Db, optimize=True)
    Fa = H + (Ja + Jb) - Ka
    Fb = H + (Ja + Jb) - Kb
    
    # Compute DIIS residual for Fa & Fb
    diis_r_a = A.dot(Fa.dot(Da).dot(S) - S.dot(Da).dot(Fa)).dot(A)
    diis_r_b = A.dot(Fb.dot(Db).dot(S) - S.dot(Db).dot(Fb)).dot(A)
    
    # Append trial & residual vectors to lists
    F_list_a.append(Fa)
    F_list_b.append(Fb)
    R_list_a.append(diis_r_a)
    R_list_b.append(diis_r_b)
    
    # Compute UHF Energy
    SCF_E = np.einsum('pq,pq->', (Da + Db), H, optimize=True)
    SCF_E += np.einsum('pq,pq->', Da, Fa, optimize=True)
    SCF_E += np.einsum('pq,pq->', Db, Fb, optimize=True)
    SCF_E *= 0.5
    SCF_E += E_nuc
    
    dE = SCF_E - E_old
    dRMS = 0.5 * (np.mean(diis_r_a**2)**0.5 + np.mean(diis_r_b**2)**0.5)
    print('SCF Iteration %3d: Energy = %4.16f dE = % 1.5E dRMS = %1.5E' % (scf_iter, SCF_E, dE, dRMS))
    
    # Convergence Check
    if (abs(dE) < E_conv) and (dRMS < D_conv):
        break
    E_old = SCF_E
    
    # DIIS Extrapolation
    if scf_iter >= 2:
        Fa = diis_xtrap(F_list_a, R_list_a)
        Fb = diis_xtrap(F_list_b, R_list_b)
    
    # Compute new orbital guess
    Ca, Da = diag_F(Fa, nalpha)
    Cb, Db = diag_F(Fb, nbeta)
    
    # MAXITER exceeded?
    if (scf_iter == MAXITER):
        psi4.core.clean()
        raise Exception("Maximum number of SCF iterations exceeded.")

# Post iterations
print('\nSCF converged.')
print('Final UHF Energy: %.8f [Eh]' % SCF_E)

==> Starting SCF Iterations <==

SCF Iteration   1: Energy = -68.9800327332966106 dE = -6.89800E+01 dRMS = 1.16551E-01
SCF Iteration   2: Energy = -69.6472544406204861 dE = -6.67222E-01 dRMS = 1.07430E-01
SCF Iteration   3: Energy = -75.7919291462228273 dE = -6.14467E+00 dRMS = 2.89274E-02
SCF Iteration   4: Energy = -75.9721892298673538 dE = -1.80260E-01 dRMS = 7.56446E-03
SCF Iteration   5: Energy = -75.9893690604500875 dE = -1.71798E-02 dRMS = 8.74982E-04
SCF Iteration   6: Energy = -75.9897163369163593 dE = -3.47276E-04 dRMS = 5.35606E-04
SCF Iteration   7: Energy = -75.9897932418062396 dE = -7.69049E-05 dRMS = 6.21200E-05
SCF Iteration   8: Energy = -75.9897956276201114 dE = -2.38581E-06 dRMS = 2.57879E-05
SCF Iteration   9: Energy = -75.9897957847445582 dE = -1.57124E-07 dRMS = 1.72817E-06

SCF converged.
Final UHF Energy: -75.98979578 [Eh]


In [23]:
from pyscf import scf
mf=scf.UHF(mol)
mf.kernel()


converged SCF energy = -75.989795787466  <S^2> = 2.6740121e-10  2S+1 = 1


-75.98979578746602